In [1]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc
from bw_temporalis import easy_timedelta_distribution, TemporalisLCA
import bw_processing as bwp
from utilities import *

# Set project and databases

In [2]:
bd.projects.set_current('tictac')

In [3]:
bd.databases

Databases dictionary with 5 object(s):
	wind-example-2020
	wind-example-2030
	wind-example-2035
	wind-example-2040
	wind-example-2050

In [4]:
def generate_simple_database(year, share_of_wind_in_electricity_mix, generated_electricity_over_lifetime=1000):
    """
    Write in the project a simple database for our testing for a given year.

    :param year: Year of the database.
    :param share_of_wind_in_electricity_mix: Share of wind electricity in the dummy electricity mix.
    :param generated_electricity_over_lifetime: Generated electricity by the wind turbine over its lifetime.
    :return: Database with the name 'wind-electricity-year'.
    """
    database_name = 'wind-example-'+str(year)
    
    if database_name in list(bd.databases):
        del bd.databases[database_name]
        
    bd.Database(database_name).write({
        (database_name, "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
            "unit": "kilogram",
        },
        (database_name, "coal"): {
            "type": "emission",
            "name": "coal",
            "unit": "kilogram",
        },
        (database_name, 'electricity-mix'): {
            'name': 'Electricity mix',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': (database_name, 'electricity-production-wind'),
                    'amount': share_of_wind_in_electricity_mix,
                    'type': 'technosphere',
                },
                {
                    'input': (database_name, 'electricity-production-coal'),
                    'amount': 1-share_of_wind_in_electricity_mix,
                    'type': 'technosphere',
                },
            ]
        },
        (database_name, 'electricity-production-coal'): {
            'name': 'Electricity production, coal',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': (database_name, 'coal'),
                    'amount': 1,
                    'type': 'biosphere',
                },
                {
                    'input': (database_name, 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                },
            ]
        },
        (database_name, 'electricity-production-wind'): {
            'name': 'Electricity production, wind',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': (database_name, 'electricity-production-wind'),
                    'amount': 1,
                    'type': 'production',
                },
                {
                    'input': (database_name, 'wind-turbine-construction'),
                    'amount': 1/generated_electricity_over_lifetime,
                    'type': 'technosphere',
                    'temporal_distribution': easy_timedelta_distribution(
                        start=-5,
                        end=0,
                        resolution="Y",  # M for months, Y for years, etc.
                        steps=6,
                    ),
                },
                # maybe add maintenance, oil changes, ...
            ]
        },
        (database_name, 'wind-turbine-construction'): {
            'name': 'Wind turbine construction',
            'unit': 'unit',
            'exchanges': [
                {
                    'input': (database_name, 'wind-turbine-construction'),
                    'amount': 1,
                    'type': 'production'
                },
                {
                    'input': (database_name, 'electricity-mix'),
                    'amount': 1,
                    'type': 'technosphere',
                },
                # {
                #     'input': (database_name, 'eol-wind'),
                #     'amount': 1,
                #     'type': 'technosphere',
                #     'temporal_distribution': easy_timedelta_distribution(
                #         start=0,
                #         end=20,
                #         resolution="Y",  # M for months, Y for years, etc.
                #         steps=10,
                #     ),
                # },
                # aggregate the rest to direct co2 emissions
                {
                    'input': (database_name, 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                }
            ]
        },
        (database_name, 'eol-wind'): {
            'name': "End-of-life, wind turbine",
            'unit': 'unit',
            'exchanges': [
                {
                    'input': (database_name, 'eol-wind'),
                    'amount': 1,
                    'type': 'production',
                },
                # aggregate the rest to direct co2 emissions
                {
                    'input': (database_name, 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                }
            ]
        },
    })

In [5]:
generate_simple_database(2020, 0.3)
generate_simple_database(2030, 0.5)
generate_simple_database(2040, 0.6)
generate_simple_database(2050, 0.7)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 7012.21it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?it/s]

Vacuuming database 


# Timeline generation calculation

### Impact assessment methods

In [6]:
bd.Method(("GWP", "wind-example-2020")).write([
    (('wind-example-2020', "CO2"), 1),
    (('wind-example-2035', "CO2"), 1),
    (('wind-example-2050', "CO2"), 1),
])

### Static LCA calculation

In [7]:
fu_2020 = {('wind-example-2020', 'electricity-production-wind'): 1}

In [8]:
lca = bc.LCA(fu_2020, ("GWP", "wind-example-2020"))
lca.lci()
lca.lcia()
lca.score

0.0017005102219358214

### Dynamic LCA with temporalis

In [9]:
tlca = TemporalisLCA(lca)

Starting graph traversal
Calculation count: 3


In [10]:
tl = tlca.build_timeline(node_timeline=True)
tl.build_dataframe()
tl_df = tl.add_metadata_to_dataframe(database_labels=["wind-example-2020"], fields=['name', 'unit'])

C:\Users\jeromea\AppData\Local\Continuum\anaconda3\envs\tictac\lib\site-packages\bw_temporalis\lca.py:129: UserWarning: This functionality is experimental, and will change.
You have been warned.
  warnings.warn(


# Patch database with prospective databases

### Information on existing databases

In [11]:
database_date_dict = {
    2020: 'wind-example-2020',
    2030: 'wind-example-2030',
    2040: 'wind-example-2040',
    2050: 'wind-example-2050',
}   

In [12]:
dates_list = [datetime.strptime(str(x), "%Y") for x in database_date_dict.keys()]

### Example of a patch

In [13]:
exchange_output = ('wind-example-2050', 'electricity-production-wind')
old_exchange_input = ('wind-example-2050', 'wind-turbine-construction')
new_exchange_input = ('wind-example-2035', 'wind-turbine-construction')
new_exchange_input_amount = 0.001

In [14]:
dp = safety_razor(
    consumer=exchange_output,
    previous_producer=old_exchange_input, 
    new_producer=new_exchange_input, 
)

In [15]:
demand = {('wind-example-2050', 'electricity-production-wind'): 1, ('wind-example-2035', 'electricity-production-wind'): 1}
ia = ("GWP", "wind-example-2020")
fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=ia)

In [16]:
lca_patched = bc.LCA(fu, data_objs=data_objs + [dp], remapping_dicts=remapping)
lca_patched.lci()
lca_patched.lcia()
lca_patched.score

0.0030015008217283224

### Generating interpolation weights

In [17]:
tl_df = add_column_interpolation_weights_on_timeline(tl_df, dates_list)
tl_df

C:\Users\jeromea\OneDrive - Chalmers\Courses\Open approaches to time in LCA\group-project\tictac_lca\Mini-Wind-Example\utilities.py:184: Warning: Date outside the range of dates covered by the databases.
  warnings.warn("Date outside the range of dates covered by the databases.", category=Warning)


,date,amount,flow,activity,activity_name,activity_unit,flow_name,flow_unit,interpolation_weights
0,2018-10-12 08:07:33,0.000167,-1,188,Wind turbine construction,unit,NaN,NaN,{2020: 1}
1,2018-10-12 08:07:33,0.000167,-1,185,Electricity mix,kilowatt hour,NaN,NaN,{2020: 1}
2,2018-10-12 08:07:33,0.000117,-1,186,"Electricity production, coal",kilowatt hour,NaN,NaN,{2020: 1}
3,2019-10-12 13:56:45,0.000167,-1,188,Wind turbine construction,unit,NaN,NaN,{2020: 1}
4,2019-10-12 13:56:45,0.000167,-1,185,Electricity mix,kilowatt hour,NaN,NaN,{2020: 1}
5,2019-10-12 13:56:45,0.000117,-1,186,"Electricity production, coal",kilowatt hour,NaN,NaN,{2020: 1}
6,2020-10-11 19:45:57,0.000167,-1,185,Electricity mix,kilowatt hour,NaN,NaN,"{2020: 0.07796977180095507, 2030: 0.9220302281..."
7,2020-10-11 19:45:57,0.000167,-1,188,Wind turbine construction,unit,NaN,NaN,"{2020: 0.07796977180095507, 2030: 0.9220302281..."
8,2020-10-11 19:45:57,0.000117,-1,186,"Electricity production, coal",kilowatt hour,NaN,NaN,"{2020: 0.07796977180095507, 2030: 0.9220302281..."
9,2021-10-12 01:35:09,0.000167,-1,188,Wind turbine construction,unit,NaN,NaN,"{2020: 0.17795403131368434, 2030: 0.8220459686..."


# Testing automated patching

In [18]:
# Creating a mock timeline

import pandas as pd
import numpy as np

# Number of rows in the DataFrame
n = 10

# Generate random dates
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2050-01-01')
date_range = (end_date - start_date).days
dates = [start_date + pd.to_timedelta(np.random.randint(0, date_range), unit='D') for _ in range(n)]

# Generate random amounts, let's say between 10 and 1000
amounts = np.random.randint(10, 1000, size=n)

# Generate random producer and consumer integers, let's say between 1 and 10
producers = np.random.randint(213, 230, size=n)
consumers = np.random.randint(213, 230, size=n)

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'amount': amounts,
    'producer': producers,
    'consumer': consumers
}).sort_values(by='date')

In [19]:
df = add_column_interpolation_weights_on_timeline(df, dates_list)

In [20]:
df

,date,amount,producer,consumer,interpolation_weights
3,2022-01-24,250,225,224,"{2020: 0.20640569395017794, 2030: 0.7935943060..."
7,2022-12-14,615,222,221,"{2020: 0.2950999178757186, 2030: 0.70490008212..."
0,2024-07-31,843,221,228,"{2020: 0.45797974267725156, 2030: 0.5420202573..."
4,2029-02-09,342,221,213,"{2020: 0.9107582808650424, 2030: 0.08924171913..."
1,2029-09-17,53,222,229,"{2020: 0.9709827539009034, 2030: 0.02901724609..."
2,2031-10-25,21,219,229,"{2030: 0.18127053669222343, 2040: 0.8187294633..."
9,2039-12-31,540,223,225,"{2030: 0.9997261774370209, 2040: 0.00027382256..."
5,2043-03-31,911,228,213,"{2040: 0.32439091157952366, 2050: 0.6756090884..."
8,2047-06-05,407,229,213,"{2040: 0.7424035039693403, 2050: 0.25759649603..."
6,2049-11-25,695,216,213,"{2040: 0.989871338625787, 2050: 0.010128661374..."


In [21]:
bd.get_activity(('wind-example-2020', 'electricity-mix')).id

185

In [22]:
def extract_edge_from_row(row):
    new_edges = []
    consumer = bd.get_node(id=row['consumer'])
    previous_producer = bd.get_node(id=row['producer'])
    for date, share in row['interpolation_weights'].items():
        database = database_date_dict[date]
        new_amount = row['amount'] * share
        new_producer = bd.get_activity(key=(database, previous_producer['code']))
        new_edges.append((consumer, previous_producer, new_producer, new_amount))
    
    return consumer

extract_edge_from_row(df.iloc[2])

# dp = safety_razor(
#     consumer=exchange_output,
#     previous_producer=old_exchange_input, 
#     new_producer=new_exchange_input, 
# )

UnknownObject: 

In [23]:
df.iloc[0]['interpolation_weights']

{2020: 0.20640569395017794, 2030: 0.7935943060498221}